In [ ]:
import findspark
import plotly.graph_objects as go
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql import SQLContext


findspark.init()
sc = SparkContext("local", "first app")
spark = SparkSession(sc)
sqlContext = SQLContext(sc)

In [ ]:
archivo_entrada = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true', quote='"', delimiter=',').load("entradas/Covid19.csv")
rddfilter = archivo_entrada.rdd.map(tuple)

In [ ]:
rddCovid= rddfilter.map(lambda word: (word[7],word[5]))
rddCovid.take(10)

In [ ]:
rddCount=rddCovid.reduceByKey(lambda a,b: a+b)
print("Conteo total -> %s" % rddCount.collect())

In [ ]:
rddOrder = rddCount.filter(lambda x: "Cuba" in x[0] or "France" in x[0] or "Canada" in x[0] or "Singapore" in x[0] or "South_Korea" in x[0])
print("Resultados filtrados-> %s"%rddOrder.collect())

In [ ]:
rddNames = rddOrder.map(lambda x: (x[0]))
rddTotals = rddOrder.map(lambda x: (x[1]))
print(rddNames.collect())
print(rddTotals.collect())

In [ ]:
fig = go.Figure(data=go.Bar(x=rddNames.collect(),y=rddTotals.collect()))
fig.update_layout(title_text='Número total de casos de COVID-19',title_font_size=30,
                  yaxis=dict(title='Número de casos',title_font_size=25),
                  xaxis=dict(title='Nombre Pais',title_font_size=25))
fig.update_traces(overwrite=True, marker={"opacity": 0.5})
fig.write_html('R1_A3.html', auto_open=True)